Import Libraries

In [1]:
!pip install tensorflow-io

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 28.0/28.0 MB 54.6 MB/s eta 0:00:00


In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tensorflow import keras
from keras.datasets import mnist
from keras import backend as k
import time
from tensorflow.keras.callbacks import EarlyStopping

# 1. Process MNIST dataset

Assign training and test data

In [4]:
batch_size = 128
num_classes = 10
img_rows, img_cols = 28,28
(x_train,y_train),(x_test,y_test) =   mnist.load_data()

11490434/11490434 [==============================] - 0s 0us/step


Reshape the images

In [5]:
if k.image_data_format()=='channels_first':
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)
else:
    x_train=x_train.reshape(x_train.shape[0],img_rows,img_cols,1)
    x_test=x_test.reshape(x_test.shape[0],img_rows,img_cols,1)

input_shape=(img_rows,img_cols,1)
x_train = x_train/255.0
x_test=x_test/255.0
print('x_train shape:',x_train.shape,'\nx_test shape:',x_test.shape)

x_train shape: (60000, 28, 28, 1) 
x_test shape: (10000, 28, 28, 1)


Convert class vectors to binary class matrices

In [6]:
y_train=keras.utils.to_categorical(y_train,num_classes)
y_test=keras.utils.to_categorical(y_test,num_classes)

# 2. CNN no Attention

Design the CNN architecture

In [7]:
from keras.models import Sequential
from keras import layers

In [8]:
model=Sequential()

model.add( layers.Conv2D(32,kernel_size=(3,3),activation='relu',input_shape=input_shape) )
model.add( layers.MaxPooling2D(pool_size=(2,2)) )
model.add( layers.Dropout(0.2) )
model.add( layers.Flatten() )
model.add( layers.Dense(32,activation='relu') )
model.add( layers.Dense(num_classes,activation='softmax') )
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 26, 26, 32)        320       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 13, 13, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 11, 11, 64)        18496     
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 5, 5, 64)         0         
 2D)                                                             
                                                                 
 flatten (Flatten)           (None, 1600)              0         
                                                                 
 dense (Dense)               (None, 32)                5

In [9]:
model.compile(optimizer=keras.optimizers.Adam(),
              loss=keras.losses.categorical_crossentropy,
              metrics=['accuracy']
             )
early_stopping = EarlyStopping(monitor='accuracy', patience=3)
tic=time.time()
model.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=(x_test,y_test)
          )
toc=time.time()
training_time=toc-tic

Epoch 1/4
469/469 [==============================] - 21s 6ms/step - loss: 0.2862 - accuracy: 0.9122 - val_loss: 0.0758 - val_accuracy: 0.9765
Epoch 2/4
469/469 [==============================] - 2s 5ms/step - loss: 0.0760 - accuracy: 0.9772 - val_loss: 0.0603 - val_accuracy: 0.9807
Epoch 3/4
469/469 [==============================] - 2s 5ms/step - loss: 0.0539 - accuracy: 0.9830 - val_loss: 0.0446 - val_accuracy: 0.9864
Epoch 4/4
469/469 [==============================] - 2s 5ms/step - loss: 0.0423 - accuracy: 0.9869 - val_loss: 0.0386 - val_accuracy: 0.9878


In [16]:
tic=time.time()
test_loss, test_acc = model.evaluate(x_test,y_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

313/313 [==============================] - 1s 2ms/step - loss: 0.0386 - accuracy: 0.9878
Test Accuracy = 98.80 %:


# 3. CNN with Attention

Design the CNN architecture

In [11]:
inputs = layers.Input(shape=input_shape)

conv1 = layers.Conv2D(32,kernel_size=(3,3),activation='relu')(inputs)
pool1 = layers.MaxPool2D(pool_size=(2,2))(conv1)
#Attention1
attention_conv1 = layers.Conv2D(1, (3,3), padding='same', activation='sigmoid')(pool1)
attention_mul1 = layers.Multiply()([pool1, attention_conv1])
pool2 = layers.MaxPool2D(pool_size=(2,2))(attention_mul1)
##########
drop1 = layers.Dropout(0.2)(pool2)
flatten2 = layers.Flatten()(drop1)
dense2 = layers.Dense(32,activation='relu')(flatten2)
dense3 = layers.Dense(num_classes,activation='softmax')(dense2)

modelAtt = keras.Model(inputs=inputs, outputs=dense3)

modelAtt.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 28, 28, 1)]  0           []                               
                                                                                                  
 conv2d_2 (Conv2D)              (None, 26, 26, 32)   320         ['input_1[0][0]']                
                                                                                                  
 max_pooling2d_2 (MaxPooling2D)  (None, 13, 13, 32)  0           ['conv2d_2[0][0]']               
                                                                                                  
 conv2d_3 (Conv2D)              (None, 13, 13, 1)    33          ['max_pooling2d_2[0][0]']        
                                                                                              

In [12]:
modelAtt.compile(optimizer=keras.optimizers.Adam(),
              loss= keras.losses.CategoricalCrossentropy(),
              metrics=['accuracy']
             )
tic=time.time()
modelAtt.fit(x_train,y_train,
          batch_size=batch_size,
          epochs=15,
          verbose=1,
          callbacks=[early_stopping],
          validation_data=(x_test,y_test)
          )
toc=time.time()
training_time=toc-tic

Epoch 1/4
469/469 [==============================] - 5s 6ms/step - loss: 0.6068 - accuracy: 0.8218 - val_loss: 0.1601 - val_accuracy: 0.9534
Epoch 2/4
469/469 [==============================] - 2s 5ms/step - loss: 0.1413 - accuracy: 0.9588 - val_loss: 0.1090 - val_accuracy: 0.9685
Epoch 3/4
469/469 [==============================] - 5s 10ms/step - loss: 0.1012 - accuracy: 0.9698 - val_loss: 0.0805 - val_accuracy: 0.9745
Epoch 4/4
469/469 [==============================] - 4s 9ms/step - loss: 0.0813 - accuracy: 0.9756 - val_loss: 0.0698 - val_accuracy: 0.9780


In [15]:
tic=time.time()
test_loss, test_acc = modelAtt.evaluate(x_test,y_test)
toc=time.time()
test_time=toc-tic
print("Training Time = {} s".format(np.round(training_time, 1)))
print("Testing Time = {} ms".format(np.round(test_time*1000, 1)))
print('Test Loss = {:.2f} %:'.format(np.round(test_loss, 3)*100))
print('Test Accuracy = {:.2f} %:'.format(np.round(test_acc, 3)*100))

313/313 [==============================] - 2s 6ms/step - loss: 0.0698 - accuracy: 0.9780
Test Accuracy = 97.80 %:
